In [1]:
#Some basic yet important and research related RS imagery download practiced using GEE platform consulting 'Earth Engine & Geemap' book by Qiusheng Wu
import ee

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
import geemap
import leafmap
import os

In [1]:
#search GEE functions, which otherwise would have to go back to GEE platform
#geemap.ee_search()

In [5]:
#Get imageries properties using GEE
#Open map and add images to the map
Map = geemap.Map(center = [40, -100], zoom = 4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [6]:
#Define a centroid point and load imageries for that area
centroid = ee.Geometry.Point([-95.65, 40.38])
Bville = ee.ImageCollection("USDA/NAIP/DOQQ").filterDate('2010-01-01', '2023-11-17')\
    .filterBounds(centroid)\
        .sort('system:time_start', False)\
            .first()

Bville_vis = {"bands": ["N","R", "G"]}
#naip_vis = {"bands": ["R", "G", "B"]}
Map.centerObject(centroid, 12)
Map.addLayer(Bville, Bville_vis, "Bville0")

In [7]:
#Get all properties all at once
naip_props = geemap.image_props(Bville)
naip_props.getInfo()

{'IMAGE_DATE': '2022-07-10',
 'NOMINAL_SCALE': 0.6,
 'system:asset_size': '495.113748 MB',
 'system:band_names': ['R', 'G', 'B', 'N'],
 'system:id': 'USDA/NAIP/DOQQ/m_4009535_se_15_060_20220710',
 'system:index': 'm_4009535_se_15_060_20220710',
 'system:time_end': '2022-07-10 00:00:00',
 'system:time_start': '2022-07-10 00:00:00',
 'system:version': 1687956899233054,
 'year': 2022}

In [9]:
#Define point and load imageries (this time to display true color imagery)
points = ee.Geometry.Point([-95.63, 40.32])
Bville2 = ee.ImageCollection("USDA/NAIP/DOQQ").filterDate('2022-01-01', '2023-12-31')\
    .filterBounds(points).first()

Bville2_vis = {"bands": ["R", "G", "B"]}
#naip_vis = {"bands": ["N", "R", "G"]}
Map.centerObject(points, 12)
Map.addLayer(Bville2, Bville2_vis, "BLA0")

In [10]:
#Get all properties all at once
naip_props = geemap.image_props(Bville2)
naip_props.getInfo()

{'IMAGE_DATE': '2022-07-10',
 'NOMINAL_SCALE': 0.6,
 'system:asset_size': '482.277856 MB',
 'system:band_names': ['R', 'G', 'B', 'N'],
 'system:id': 'USDA/NAIP/DOQQ/m_4009543_ne_15_060_20220710',
 'system:index': 'm_4009543_ne_15_060_20220710',
 'system:time_end': '2022-07-10 00:00:00',
 'system:time_start': '2022-07-10 00:00:00',
 'system:version': 1687962137115189,
 'year': 2022}

In [11]:
#load two Bville NAIP imageries using their unique system id and mosaic  as one image
Bville01 = ee.Image('USDA/NAIP/DOQQ/m_4009535_se_15_060_20220710').select(['R', 'G', 'B', 'N'])
Bville02 = ee.Image('USDA/NAIP/DOQQ/m_4009543_ne_15_060_20220710').select(['R', 'G', 'B', 'N'])

#Mosaic
mosaic_Bville = ee.ImageCollection([Bville01, Bville02]).mosaic()

In [12]:
#says successful but didn't work to save the mosiaiced imagery to Google Drive; so I downloaded each tile separately in my local working folder
geemap.ee_export_image_to_drive(
    Bville01, description='naip', folder='GEE_Exports', scale=0.6
)

In [14]:
#Download a single image in a local drive (working directory set in the Visual Studio Code)
geemap.download_ee_image(Bville01, filename='bv1_naip.tif', scale=0.6)

bv1_naip.tif: |          | 0.00/523M (raw) [  0.0%] in 00:00 (eta:     ?)

In [15]:
print(type(mosaic_Bville))

<class 'ee.image.Image'>


In [16]:
#visualization of mosaicked imagery, false color (one tile of true color and another of false color imagery is already displayed above separately)
# Set visualization parameters
vis_params = {
    'bands': ['N', 'R', 'G']  # Can also use RGB bands
    }

# Create a map and display the mosaic
Map = geemap.Map(center = [40.38, -95.65], zoom = 14)
Map.addLayer(mosaic_Bville, vis_params, "Mosaic_Bville")
Map.addLayerControl()  # Add layer control to the map
Map

Map(center=[40.38, -95.65], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [18]:
#Set the directory to load shapefile and to download the imageries
filepath = r'Z:\NAIP\eeNAIP\Bville'
#print(filepath)

In [19]:
#read .shp file, covert to ee feature and then add to the map to ensure it is there :)
shpfile = os.path.join(filepath, 'Brownville_buf.shp')
shapefile_ee = geemap.shp_to_ee(shpfile)
Map.addLayer(shapefile_ee, {}, "Buffer Layer")

In [20]:
#Clip NAIP and vizualize to ensure its done adequately
clipped_Bville = mosaic_Bville.clip(shapefile_ee)

#add to map using same visual parameters
Map.addLayer(clipped_Bville, vis_params, "Clipped_NAIP")

In [22]:
#check the class of the mosaiced and then clipped Naip imagery to Bville area
print(type(clipped_Bville))

<class 'ee.image.Image'>


In [23]:
#check the projection information of the mosaiced and then clipped imagery
projection = mosaic_Bville.select(0).projection().getInfo()
projection

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}